# Downloading NLDAS Data

[THIS CODE IS ONLY SLIGHTLY ADJUSTED FROM A SCRIPT WRITTEN BY KEVIN SCHARZWALD]

NLDAS files are available in hourly timesteps: ERA5 files are available in hourly timesteps; this code allows for resampling the data at lower frequencies (see the section on [Subsetting and Resampling](#subset)).


....



Files are placed in a subdirectory of the `raw_data_dir` set in the `dwnld_config.py` file `[raw_data_dir]/ERA5/`. If this subdirectory doesn't yet exist, it is created. Output files have the following filename: 

`[raw_data_dir]/NLDAS/[short_name]_[output_freq_name]_NLDAS_[MODEL and/or FORCING]_[allyears[0]0101-allyears[-1]1231]_[fn_suffix].nc` 

with `[raw_data_dir]` set from the `config` file, `[short_name]` set in the `download_vars`, `output_freq_name` set in the `resampling_vars` (unless no time resampling is conducted; in this case `_hr_` is used), and `fn_suffix` set in the `geographic_vars`. 

# Setup

## Packages

In [10]:
import xarray as xr
import numpy as np
import datetime as dt
import os
from tqdm.notebook import tqdm
import re
%matplotlib inline



In [4]:
# Get config file 
import dwnld_config as cfg

In [5]:
# Set years to process
all_years = np.arange(1979,2015)
#all_years = np.arange(1979, 1980)

# How many years to be processed at a time (may impact performance; detailed checks forthcoming)
chunk_size = 5

GLDAS data is released as hourly data. This code supports resampling using the `xarray.Dataset.resample()` function ([docs here](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.resample.html)), which is based on the `pandas.resample()` functionality. The next cell adjusts whether data is resampled. 

The `resample_level` value is piped into the `freq` call of the `.resample` indexer. Useful, common options include: 
- `D`: daily
- `M`: month (end)
- `MS`: month (beginning)

A full list of options is available in the `pandas` documentation [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects).

In [6]:
# Set resampling
resampling_vars = {'resamp':True, # whether to resample from hourly (if false, the rest is ignored)
                   'resample_level':'D', # what level to resample to 
                   'output_freq_name':'day'} # what the frequency should be called in the filename, attributes

In [7]:
cfg.lpaths['raw_data_dir']

'/Users/miriam/Documents/02_academia/02_PhD/research/raw_data/'

# Downloading 


In [8]:
if not os.path.exists(cfg.lpaths['raw_data_dir']):
    os.mkdir(cfg.lpaths['raw_data_dir'])
    print(cfg.lpaths['raw_data_dir']+' created')